In [1]:
from osgeo import ogr
import json
import pyproj
import pandas as pd
from shapely.geometry import shape
from shapely.geometry.polygon import Polygon, Point
from shapely.ops import transform
from math import inf, pi, cos
from haversine import Unit, haversine

Constantes

In [2]:
MAPS_PATHS = [
    './data-insivumeh/MapasGuatemala/Fisiografia/FISIOGRAFIAFINAL_GTM.shp',
    './data-insivumeh/MapasGuatemala/Geología/GEOLOGIA_GTM.shp',
    './data-insivumeh/MapasGuatemala/Suelo Nivel Reconocimiento/SSUELOS_GTM.shp',
    './data-insivumeh/MapasGuatemala/Taxonomia Suelos/Shapefile Taxonomia a nivel de Orden Nacional/TAXONOMIA_ORDENES_GTM.shp',
    './data-insivumeh/MapasGuatemala/Uso de Suelo/USDA_GTM.shp'
]
GT_MAP_PATH = './data-insivumeh/IGN/mapa_guatemala/gadm41_GTM_0.shp'
RES_DATA = './results/data_insivumeh_all_gt_map.csv'

In [3]:
class DataGenerator:
    properties = {
        'GEO': 'rock_geo',
        'EDADGEOGP': 'rock_period',
        'NOMR': 'fis_nomr',
        'TIPO_ROCA': 'rock_type',
        'SIMBOLO': 'soil_rec_symbol',
        'MAT_ORIG': 'soil_rec_mat_origin',
        'CLAS_DRE': 'soil_rec_sew_class',
        'TEX_SUP': 'soil_rec_tex_sup',
        'C_TEXSUP': 'soil_rec_c_tex_sup',
        'TEX_SUB': 'soil_rec_tex_sub',
        'C_TEXSUB': 'soil_rec_c__tex_sub',
        'RIES_ERO': 'soil_rec_risk_eros',
        'ORDEN': 'soil_tax',
        'CAPACIDAD': 'soil_use'
    }
    def __init__(self, gt_map_directory, maps_directories):
        self.maps = dict()
        self.count = dict()
        target = pyproj.CRS('EPSG:4326')

        layer, features = self.__getmap__(gt_map_directory, target)
        self.map = features[0]


        for directory in maps_directories:
            print('Cargando:', directory)
            file = ogr.Open(directory)
            layer = file.GetLayer()
            source = pyproj.CRS(layer.GetSpatialRef().GetName())
            features =  [json.loads(layer.GetFeature(i).ExportToJson()) for i in range(layer.GetFeatureCount())]
            project = pyproj.Transformer.from_crs(source, target, always_xy=True).transform if source != target else None
            
            for feature in features:  
                if project is None:
                    feature['geometry'] = shape(feature['geometry'])
                else:
                    feature['geometry'] = transform(project, shape(feature['geometry']))
            
            self.maps[layer.GetName()] =  features
            self.count[layer.GetName()] =  0
        

    def __getmap__(self, directory, target):
        print('Cargando:', directory)
        file = ogr.Open(directory)
        layer = file.GetLayer()
        source = pyproj.CRS(layer.GetSpatialRef().GetName())
        features =  [json.loads(layer.GetFeature(i).ExportToJson()) for i in range(layer.GetFeatureCount())]
        project = pyproj.Transformer.from_crs(source, target, always_xy=True).transform if source != target else None
            
        for feature in features:  
            if project is None:
                feature['geometry'] = shape(feature['geometry'])
            else:
                feature['geometry'] = transform(project, shape(feature['geometry']))

        return layer, features
    
    def __getmapsinformation__(self, point):  
        dictData = dict() 
        for map_name in self.maps.keys():
                properties = self.__getmapproperties__(map_name, point)
                if properties:   
                    dictData.update(properties)
                else: 
                    self.count[map_name] += 1
        return dictData

    def __getmapproperties__(self, map_name: str, point):   
        for feature in self.maps[map_name]:  
            polygon: Polygon = feature['geometry']
            if polygon.contains(point):
                return {self.properties[k]: v for k, v in feature['properties'].items() if k in self.properties.keys()}
    
    def __getisinmap__(self, point):   
        polygon: Polygon = self.map['geometry']
        if polygon.contains(point):
            return True
        else:
            return False

    def __getmapbounds__(self):   
        polygon: Polygon = self.map['geometry']
        return polygon.bounds
    
    def __getnewlnglat__(self, lnglat, dy, dx):   
        r_earth = 6378100
        latitude = lnglat[1]
        longitude = lnglat[0] 
        new_latitude  = latitude  + (dy / r_earth) * (180 / pi)
        new_longitude = longitude + (dx / r_earth) * (180 / pi) / cos(latitude * pi/180)
        # print(haversine((longitude, latitude),(new_longitude, new_latitude),unit=Unit.METERS, normalize=True))
        return [new_longitude, new_latitude]

    def getData(self):
        output = pd.DataFrame()
        bounds = self.__getmapbounds__()
        lnglat = bounds[:2]
        landslide_id = 0
        while lnglat[1] < bounds[3]:
            while lnglat[0] < bounds[2]:
                point = Point(lnglat[0], lnglat[1])
                if self.__getisinmap__(point):
                    landslide_id += 1
                    dictData = dict()
                    dictData['landslide_id'] = landslide_id
                    dictData['longitude'] = lnglat[0]
                    dictData['latitude'] = lnglat[1]
                    dictData.update(self.__getmapsinformation__(point))
                    output = pd.concat([output, pd.DataFrame(dictData, index=[dictData['landslide_id']])])
                lnglat = self.__getnewlnglat__(lnglat, 0, 1000)
            lnglat[0] = bounds[0]
            lnglat = self.__getnewlnglat__(lnglat, 1000, 0)
            
        for map_name in self.maps.keys():
            print('Cantidad de puntos con propiedades del mapa "{}" faltantes: {}'.format(map_name, self.count[map_name]))
        return output            

In [4]:
dg = DataGenerator(GT_MAP_PATH, MAPS_PATHS)
data = dg.getData()

Cargando: ./data-insivumeh/IGN/mapa_guatemala/gadm41_GTM_0.shp
Cargando: ./data-insivumeh/MapasGuatemala/Fisiografia/FISIOGRAFIAFINAL_GTM.shp
Cargando: ./data-insivumeh/MapasGuatemala/Geología/GEOLOGIA_GTM.shp
Cargando: ./data-insivumeh/MapasGuatemala/Suelo Nivel Reconocimiento/SSUELOS_GTM.shp
Cargando: ./data-insivumeh/MapasGuatemala/Taxonomia Suelos/Shapefile Taxonomia a nivel de Orden Nacional/TAXONOMIA_ORDENES_GTM.shp
Cargando: ./data-insivumeh/MapasGuatemala/Uso de Suelo/USDA_GTM.shp


In [ ]:
data.to_csv(RES_DATA)  